In [22]:
import json
import os
import pandas as pd
import datetime
import xlsxwriter
import xml.etree.ElementTree as ET
import csv

workbook = xlsxwriter.Workbook("IntersectionByHourRO.xlsx")
bold = workbook.add_format({'bold': True})
red  = workbook.add_format({'font_color': 'red'})
worksheet1 = workbook.add_worksheet()
r= 0
worksheet1.write(0, 0, "Hour", bold)
while r<24:
    worksheet1.write(r+1, 0, r)
    r=r+1

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]
    if(line1[0][0]==line1[1][0] and line2[0][0]==line2[1][0] and line1[0][0]==line2[1][0]):
        return line1[0][0], (line1[0][1]+line2[1][1])/2
    if(line1[0][1]==line1[1][1] and line2[0][1]==line2[1][1] and line1[0][1]==line2[1][1]):
        return (line1[0][0]+line2[1][0])/2, line1[0][1]
    div = det(xdiff, ydiff)
    if div == 0:
        # print("Differnece: ",xdiff, ydiff)
        return 0, 0
    #    raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y


def onSegment(p, q, r): 
    if ( (q[0] <= max(p[0], r[0])) and (q[0] >= min(p[0], r[0])) and 
           (q[1] <= max(p[1], r[1])) and (q[1] >= min(p[1], r[1]))): 
        return True
    return False
  
def orientation(p, q, r): 
    # 0 : Collinear points 
    # 1 : Clockwise points 
    # 2 : Counterclockwise 
      
    val = (float(q[1] - p[1]) * (r[0] - q[0])) - (float(q[0] - p[0]) * (r[1] - q[1])) 
    if (val > 0):  
        # Clockwise orientation 
        return 1
    elif (val < 0): 
        # Counterclockwise orientation 
        return 2
    else: 
        # Collinear orientation 
        return 0
  
def intersect(p1,q1,p2,q2): 
    o1 = orientation(p1, q1, p2) 
    o2 = orientation(p1, q1, q2) 
    o3 = orientation(p2, q2, p1) 
    o4 = orientation(p2, q2, q1) 
  
    # General case 
    if ((o1 != o2) and (o3 != o4)): 
        return True
  
    # Special Cases 
  
    # p1 , q1 and p2 are collinear and p2 lies on segment p1q1 
    if ((o1 == 0) and onSegment(p1, p2, q1)): 
        return True
  
    # p1 , q1 and q2 are collinear and q2 lies on segment p1q1 
    if ((o2 == 0) and onSegment(p1, q2, q1)): 
        return True
  
    # p2 , q2 and p1 are collinear and p1 lies on segment p2q2 
    if ((o3 == 0) and onSegment(p2, p1, q2)): 
        return True
  
    # p2 , q2 and q1 are collinear and q1 lies on segment p2q2 
    if ((o4 == 0) and onSegment(p2, q1, q2)): 
        return True
  
    # If none of the cases 
    return False
date = datetime.datetime.now()
date = date - datetime.timedelta(days=0)
df = pd.DataFrame()
# df.insert(0,"Hour",range(int(24)))
columnNum=0
# fileName ="2024-3-10_22.json"
flagInt=0
h=23
while flagInt<28:
    
    
    day=str(date.year)+'-'+str(date.month)+'-'+str(date.day)
    
    dfColumn =str(date.year)+'-'+str(date.month)+'-'+str(date.day) 
    fileName =str(day)+'.csv'
    path = './'+str(day)+'.csv'
    
    
    sellD = []
    buyD = []
   
    

    if(dfColumn in df.columns)==False:
        df.insert(columnNum,dfColumn,'NA')
        columnNum= columnNum+1
        worksheet1.write(0, columnNum, dfColumn, bold)
    dfRow=h
    h=h-1
    if(h==-1):
        h=23
        date = date - datetime.timedelta(days=1)
    
    if(os.path.isfile(path)==False):
        flagInt= flagInt+1
        print(path)
        continue
    
    with open(str(fileName), 'r') as csvfile:
  # Create a reader object
        csv_reader = csv.reader(csvfile)
        
        # Iterate through the rows in the CSV file
        for row in csv_reader:
            # Access each element in the row
            if(str(row[1])==str(dfRow) and str(row[2])=='sellD'):
                sellD.append({'x': float(row[3]), 'y':float(row[4])})
            if(str(row[1])==str(dfRow) and str(row[2])=='buyD'):
                buyD.append({'x': float(row[3]), 'y':float(row[4])})
        # buyD.reverse()

    i=1
    try:
        lastX = sellD[0]['x']
        lastY = sellD[0]['y']
        lastXB = buyD[0]['x']
        lastYB = buyD[0]['y']
        
    except:
        worksheet1.write(dfRow+1, columnNum, "Няма данни за часа")
        continue
    sRight=1
    bRight=0
    if(lastX<=sellD[7]['x']):sRight=1
    else:sRight=0
    if(lastXB<buyD[7]['x']):bRight=1
    else:bRight=0
    flagW =0
    strogoPrediS=0
    strogoPrediB=0
    closestS=1
    closestB=1
    while i< len(sellD): 
        j=1
        if(sRight==1 and bRight==0 or sRight==1 and bRight==1):
            while j< len(buyD):
                if(sellD[i]['x']<buyD[j]['x'] and sellD[i]['y']<buyD[j]['y']and flagW ==0 ): strogoPrediS=i
                    
                if(sellD[i]['x']>buyD[j]['x'] and sellD[i]['y']>buyD[j]['y']and flagW ==0 ):
                    closestS=i
                    closestB=j
                    flagW =1
                    
                j=j+1
                if(flagW==1):break
     
            i=i+1
        else:
           
            while j< len(buyD):
                if(sellD[i]['x']>buyD[j]['x'] and sellD[i]['y']>buyD[j]['y']and flagW ==0 ): strogoPrediS=i
                if(sellD[i]['x']<buyD[j]['x'] and sellD[i]['y']<buyD[j]['y']and flagW ==0 ):
                    closestS=i
                    closestB=j
                    flagW =1
             
                j=j+1
                if(flagW==1):break
           
            i=i+1
    #rezerven cikul 

    flagW =0
    i=1
    while i< len(buyD):
        j=1
        if(bRight==0):
            while j< len(sellD):
                if(sellD[j]['x']<buyD[i]['x'] and sellD[j]['y']>buyD[i]['y']and flagW ==0 ): strogoPrediB=i
                if(sellD[j]['x']>buyD[i]['x'] and sellD[j]['y']<buyD[i]['y']and flagW ==0 ):
                    closestB=i
                    flagW =1
                j=j+1
                if(flagW==1):break
        else:
            while j< len(sellD):
                if(sellD[j]['x']>buyD[i]['x'] and sellD[j]['y']<buyD[i]['y']and flagW ==0 ): strogoPrediB=i
                if(sellD[j]['x']<buyD[i]['x'] and sellD[j]['y']>buyD[i]['y']and flagW ==0 ):
                    closestB=i
                    flagW =1
                j=j+1
                if(flagW==1):break
        i=i+1


    # krai na rezervniq cikul  

    #nachalo strogo predi strogo sled
    # print("Razlika mejud dvete strogi tochki sell")
    # print(closestS-strogoPrediS)
    A= (sellD[closestS]['x'],sellD[closestS]['y'])
    B= (sellD[closestS-1]['x'],sellD[closestS-1]['y'])
    C= (buyD[closestB-1]['x'],buyD[closestB-1]['y'])
    D= (buyD[closestB]['x'],buyD[closestB]['y'])
    # print(line_intersection((A, B), (C, D)))
    F= (1,2)
    F= line_intersection((A, B), (C, D))
    
    if(closestS-strogoPrediS>1):
        k=closestS-strogoPrediS
        while k>1:
            # print(round(F[0],2),round(A[0],2),round(F[1],2),round(A[1],2))
            if(round(F[0],5)>round(A[0],5) or round(F[1],5)>round(A[1],5)):
                closestS= closestS+1
                # print("dobavqne kum closestS")
            elif(round(F[0],5)<round(B[0],5) or round(F[1],5)<round(B[1],5)): 
                closestS= closestS-1
                # print("izvajdane ot closestS")
            A= (sellD[closestS]['x'],sellD[closestS]['y'])
            B= (sellD[closestS-1]['x'],sellD[closestS-1]['y'])
            F= line_intersection((A, B), (C, D))
            # print(line_intersection((A, B), (C, D)))
            k=k-1

    # print("Razlika mejud dvete strogi tochki buy")
    # print(closestB-strogoPrediB)       
    if(closestB-strogoPrediB>1):
        k=closestB-strogoPrediB
        while k>1:
            if(round(F[0],5)<round(D[0],5) or round(F[1],5)>round(D[1],5)):
                closestB= closestB+1
                # print("dobavqne kum closestB")
                
            elif(round(F[0],5)>round(C[0],5) or round(F[1],5)<round(C[1],5)): 
                closestB= closestB-1
                # print("izvajdane ot closestB")
            C= (buyD[closestB-1]['x'],buyD[closestB-1]['y'])
            D= (buyD[closestB]['x'],buyD[closestB]['y'])
            F= line_intersection((A, B), (C, D))
            # print(F)
            k=k-1

    #krai strogo predi strogo sled

    A= (sellD[closestS]['x'],sellD[closestS]['y'])
    B= (sellD[closestS-1]['x'],sellD[closestS-1]['y'])
    C= (buyD[closestB-1]['x'],buyD[closestB-1]['y'])
    D= (buyD[closestB]['x'],buyD[closestB]['y'])
   
    # print("Sell closest")
    # print(sellD[closestS]['x'],sellD[closestS]['y'])
    # print("Buy closest")
    # print(buyD[closestB]['x'],buyD[closestB]['y'])
    result = line_intersection((A, B), (C, D))
    if(intersect(A,B,C,D)==False):
        text = "!!"+str(round(result[0],2))+"/ "+str(round(result[1],2))+"!!"
        worksheet1.write(dfRow+1, columnNum, text, red)
        # print(result, intersect(A,B,C,D), fileName, dfRow)
        
        # flagInt= flagInt+1
        continue
    
    # print(round(result[0],2),round(result[1],2))
    # df.at[dfRow, dfColumn] = str(round(result[0],2))+"/ "+str(round(result[1],2))
    # print(result, intersect(A,B,C,D), fileName, dfRow)

    text = str(round(result[0],2))+"/ "+str(round(result[1],2))
    worksheet1.write(dfRow+1, columnNum, text)
    # flagInt= flagInt+1
    # print(result)
    
# df.sort_values("Hour")
# print(df) 
worksheet1.write(25, 0, "Легенда: при оцветена и оградена с удивителни знаци стойност, няма достатъчно данни за се стигне до пресичане.", bold)
worksheet1.write(26, 0, "В тези клетки стоиностите са само прожекции и ако са две нули не се пресичат прожекциите.", bold)
worksheet1.set_column(1, columnNum, 20)
workbook.close()


C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(columnNum,dfColumn,'NA')
C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(columnNum,dfColumn,'NA')
C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-14.csv
./2024-2-13.csv
./2024-2-13.csv
./2024-2-13.csv
./2024-2-13.csv


C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(columnNum,dfColumn,'NA')
C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(columnNum,dfColumn,'NA')
C:\Users\marketanalysis\AppData\Local\Temp\ipykernel_17240\402333341.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa